In [1]:
import tensorflow as tf
import numpy as np
import setGPU

setGPU: Setting GPU to: 0


In [2]:
tf.__version__

'2.2.0'

In [3]:
t = tf.range(0., 10.)
print(t)
t = t * 2
print(t)
t = t + 1
print(t)

tf.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(10,), dtype=float32)
tf.Tensor([ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.], shape=(10,), dtype=float32)
tf.Tensor([ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.], shape=(10,), dtype=float32)


In [4]:
@tf.function
def my_func(t):
    print("new graph created", t.dtype, t.shape)
    t = t * 2
    print(t)
    t = t + 1
    print(t)
    return t

t = tf.range(0., 10.)
my_func(t)

new graph created <dtype: 'float32'> (10,)
Tensor("mul:0", shape=(10,), dtype=float32)
Tensor("add:0", shape=(10,), dtype=float32)


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.], dtype=float32)>

In [5]:
print(t.numpy())

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [6]:
my_func(tf.range(10., 20.))

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([21., 23., 25., 27., 29., 31., 33., 35., 37., 39.], dtype=float32)>

In [7]:
my_func(tf.range(10., 21.))
print("---")
my_func(tf.range(10, 20, dtype=tf.int32))

new graph created <dtype: 'float32'> (11,)
Tensor("mul:0", shape=(11,), dtype=float32)
Tensor("add:0", shape=(11,), dtype=float32)
---
new graph created <dtype: 'int32'> (10,)
Tensor("mul:0", shape=(10,), dtype=int32)
Tensor("add:0", shape=(10,), dtype=int32)


<tf.Tensor: shape=(10,), dtype=int32, numpy=array([21, 23, 25, 27, 29, 31, 33, 35, 37, 39], dtype=int32)>

In [36]:
import dadrah.playground.test_gradient_tape as tegrta

In [37]:
x = tf.random.uniform([3000, 1], minval=1, maxval=100, dtype=tf.float32)

In [38]:
x.shape

TensorShape([3000, 1])

In [39]:
from importlib import reload
reload(tegrta)
model = tegrta.make_model(x_mean_var=(np.mean(x), np.var(x)))
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
feature_normalization_1 (Fea (None, 1)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                40        
_________________________________________________________________
dense_5 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_6 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_____________________________________________________________

In [40]:
optimizer = tf.keras.optimizers.Adam(0.005)

In [41]:
y = tf.random.uniform([3000,1], minval=1, maxval=10, dtype=tf.float32)

In [42]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(128)

In [43]:
quantile_loss = tegrta.make_quantile_loss(np.mean(y), np.var(y))

In [44]:
epochs = 30
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            predictions = model(x_batch_train, training=True)
            loss_value = quantile_loss(y_batch_train, predictions)
        
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 2 == 0:
            print("Training loss (for one batch) at step {}: {}".format(step, np.sum(loss_value)))
            print("Seen so far: %s samples" % ((step + 1) * 32))



Start of epoch 0
Training loss (for one batch) at step 0: 18.58209991455078
Seen so far: 32 samples
Training loss (for one batch) at step 2: 21.166526794433594
Seen so far: 96 samples
Training loss (for one batch) at step 4: 15.412264823913574
Seen so far: 160 samples
Training loss (for one batch) at step 6: 12.009082794189453
Seen so far: 224 samples
Training loss (for one batch) at step 8: 11.298194885253906
Seen so far: 288 samples
Training loss (for one batch) at step 10: 8.151603698730469
Seen so far: 352 samples
Training loss (for one batch) at step 12: 7.299548149108887
Seen so far: 416 samples
Training loss (for one batch) at step 14: 8.664018630981445
Seen so far: 480 samples
Training loss (for one batch) at step 16: 8.327523231506348
Seen so far: 544 samples
Training loss (for one batch) at step 18: 8.19874382019043
Seen so far: 608 samples
Training loss (for one batch) at step 20: 7.93535041809082
Seen so far: 672 samples
Training loss (for one batch) at step 22: 7.60047340

Training loss (for one batch) at step 18: 7.5203375816345215
Seen so far: 608 samples
Training loss (for one batch) at step 20: 7.740170001983643
Seen so far: 672 samples
Training loss (for one batch) at step 22: 7.392614364624023
Seen so far: 736 samples

Start of epoch 9
Training loss (for one batch) at step 0: 8.204527854919434
Seen so far: 32 samples
Training loss (for one batch) at step 2: 7.47432804107666
Seen so far: 96 samples
Training loss (for one batch) at step 4: 7.837993144989014
Seen so far: 160 samples
Training loss (for one batch) at step 6: 8.14365005493164
Seen so far: 224 samples
Training loss (for one batch) at step 8: 7.800989151000977
Seen so far: 288 samples
Training loss (for one batch) at step 10: 7.121391296386719
Seen so far: 352 samples
Training loss (for one batch) at step 12: 7.294346332550049
Seen so far: 416 samples
Training loss (for one batch) at step 14: 7.644660949707031
Seen so far: 480 samples
Training loss (for one batch) at step 16: 7.38343715667

Training loss (for one batch) at step 10: 7.132172584533691
Seen so far: 352 samples
Training loss (for one batch) at step 12: 7.296083450317383
Seen so far: 416 samples
Training loss (for one batch) at step 14: 7.645163536071777
Seen so far: 480 samples
Training loss (for one batch) at step 16: 7.38758659362793
Seen so far: 544 samples
Training loss (for one batch) at step 18: 7.503411293029785
Seen so far: 608 samples
Training loss (for one batch) at step 20: 7.737621307373047
Seen so far: 672 samples
Training loss (for one batch) at step 22: 7.391625881195068
Seen so far: 736 samples

Start of epoch 18
Training loss (for one batch) at step 0: 8.197528839111328
Seen so far: 32 samples
Training loss (for one batch) at step 2: 7.48165225982666
Seen so far: 96 samples
Training loss (for one batch) at step 4: 7.831361770629883
Seen so far: 160 samples
Training loss (for one batch) at step 6: 8.13746166229248
Seen so far: 224 samples
Training loss (for one batch) at step 8: 7.808472156524

Training loss (for one batch) at step 16: 7.389687538146973
Seen so far: 544 samples
Training loss (for one batch) at step 18: 7.497355937957764
Seen so far: 608 samples
Training loss (for one batch) at step 20: 7.73539924621582
Seen so far: 672 samples
Training loss (for one batch) at step 22: 7.387892246246338
Seen so far: 736 samples

Start of epoch 26
Training loss (for one batch) at step 0: 8.203187942504883
Seen so far: 32 samples
Training loss (for one batch) at step 2: 7.474992752075195
Seen so far: 96 samples
Training loss (for one batch) at step 4: 7.841770648956299
Seen so far: 160 samples
Training loss (for one batch) at step 6: 8.151445388793945
Seen so far: 224 samples
Training loss (for one batch) at step 8: 7.786258697509766
Seen so far: 288 samples
Training loss (for one batch) at step 10: 7.110982894897461
Seen so far: 352 samples
Training loss (for one batch) at step 12: 7.294792652130127
Seen so far: 416 samples
Training loss (for one batch) at step 14: 7.6430544853

In [45]:
x_test = tf.random.uniform([300, 1], minval=1, maxval=100, dtype=tf.float32)

In [46]:
x_test

<tf.Tensor: shape=(300, 1), dtype=float32, numpy=
array([[23.52121  ],
       [ 5.6895785],
       [40.299137 ],
       [24.915867 ],
       [11.131139 ],
       [42.03452  ],
       [72.43971  ],
       [ 8.1663   ],
       [ 5.486365 ],
       [99.60236  ],
       [ 5.014969 ],
       [89.82392  ],
       [81.182976 ],
       [62.763287 ],
       [34.006424 ],
       [ 5.039127 ],
       [14.0648575],
       [97.17816  ],
       [98.78617  ],
       [ 9.456794 ],
       [75.49267  ],
       [13.146813 ],
       [ 6.9719534],
       [10.219362 ],
       [ 9.137392 ],
       [44.76474  ],
       [52.38588  ],
       [57.040207 ],
       [46.433693 ],
       [28.694235 ],
       [62.02917  ],
       [85.37827  ],
       [44.421913 ],
       [57.02465  ],
       [61.623287 ],
       [92.22365  ],
       [74.09047  ],
       [12.265213 ],
       [28.42827  ],
       [39.809566 ],
       [36.44872  ],
       [62.145725 ],
       [74.97686  ],
       [22.868351 ],
       [91.095474 ],
     

In [47]:
y = model.predict(x_test)

In [48]:
y

array([[0.5703129 ],
       [0.57280207],
       [0.5679818 ],
       [0.57011825],
       [0.57204247],
       [0.56776875],
       [0.5654014 ],
       [0.5724563 ],
       [0.57283044],
       [0.56393886],
       [0.57289624],
       [0.5642747 ],
       [0.5647346 ],
       [0.5661392 ],
       [0.5688555 ],
       [0.57289284],
       [0.571633  ],
       [0.5640221 ],
       [0.56396693],
       [0.5722762 ],
       [0.5651685 ],
       [0.5717611 ],
       [0.5726231 ],
       [0.5721698 ],
       [0.5723208 ],
       [0.5675191 ],
       [0.5669347 ],
       [0.5665778 ],
       [0.56739116],
       [0.56959313],
       [0.56619525],
       [0.5644275 ],
       [0.5675455 ],
       [0.5665789 ],
       [0.5662262 ],
       [0.56419235],
       [0.56527543],
       [0.57188416],
       [0.5696301 ],
       [0.5680497 ],
       [0.56851643],
       [0.5661863 ],
       [0.56520784],
       [0.57040405],
       [0.56423104],
       [0.57337195],
       [0.56394553],
       [0.564

In [49]:
weights = model.get_weights()

In [50]:
model.save('./my_model.h5')

In [51]:
reload(tegrta)
loaded_model = tf.keras.models.load_model('./my_model.h5', custom_objects={'FeatureNormalization': tegrta.FeatureNormalization})

In [52]:
loaded_weights = loaded_model.get_weights()

In [53]:
weights[0]

array([[-0.26898333, -0.01164479,  0.16062233, -0.18270159,  0.14628926,
         0.2748221 , -0.00174278,  0.180469  , -0.504072  ,  0.2314341 ,
        -0.35277727,  0.29731816, -0.05552946, -0.19825546, -0.2693522 ,
        -0.47772452, -0.04418206, -0.4411161 ,  0.22911055,  0.06180804]],
      dtype=float32)

In [54]:
loaded_weights[0]

array([[-0.26898333, -0.01164479,  0.16062233, -0.18270159,  0.14628926,
         0.2748221 , -0.00174278,  0.180469  , -0.504072  ,  0.2314341 ,
        -0.35277727,  0.29731816, -0.05552946, -0.19825546, -0.2693522 ,
        -0.47772452, -0.04418206, -0.4411161 ,  0.22911055,  0.06180804]],
      dtype=float32)

In [55]:
for i in range(len(weights)):
    assert np.allclose(weights[0], loaded_weights[0])

In [56]:
y_loaded = loaded_model.predict(x_test)

In [57]:
assert np.allclose(y, y_loaded)